In [72]:
import pandas as pd
import ast
import numpy as np
from pathlib import Path  
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Patch
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.colors as mcolors

import os
# os.environ["OMP_NUM_THREADS"] = "2"

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV


from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error

#import plotly.express as px
import urllib.request
import zipfile
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter

from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity



import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak")


# Load the Excel file
df = pd.read_csv('../../data/raw/msa-municipality.csv')
HDD = pd.read_csv('../../data/raw/heating_degree_days.csv',skiprows=4)

# Create a new dataframe
data = []

# Iterate through each row in the original dataframe
for index, row in df.iterrows():
    msa = row.iloc[0]  # Get the MSA
    counties = ast.literal_eval(row.iloc[1])  # Convert the string representation of the list back to a list
    
    # Append each county with its corresponding MSA to the new data list
    for county in counties:
        data.append([county, msa])

# Create a new dataframe from the data list
msa_county = pd.DataFrame(data, columns=['County', 'MSA'])

# Separate county and state
msa_county [['County Name', 'State']] = msa_county ['County'].str.split(', ', expand=True)
msa_county  = msa_county[['County Name', 'State', 'MSA']]

# Remove the word "County" from the "County Name" column
msa_county['County Name'] = msa_county['County Name'].str.replace('County', '').str.strip()


msa_county

,County Name,State,MSA
0,Callahan,TX,"Abilene, TX"
1,Jones,TX,"Abilene, TX"
2,Taylor,TX,"Abilene, TX"
3,Portage,OH,"Akron, OH"
4,Summit,OH,"Akron, OH"
...,...,...,...
1176,Trumbull,OH,"Youngstown-Warren-Boardman, OH-PA"
1177,Mercer,PA,"Youngstown-Warren-Boardman, OH-PA"
1178,Sutter,CA,"Yuba City, CA"
1179,Yuba,CA,"Yuba City, CA"


In [73]:
HDD

,ID,Name,State,Value,Anomaly (1901-2000 base period),Rank,1901-2000 Mean,Unnamed: 7
0,AL-001,Autauga County,Alabama,1862,-594,5,2456,NaN
1,AL-003,Baldwin County,Alabama,1293,-472,4,1765,NaN
2,AL-005,Barbour County,Alabama,1788,-436,9,2224,NaN
3,AL-007,Bibb County,Alabama,2272,-481,11,2753,NaN
4,AL-009,Blount County,Alabama,2585,-646,8,3231,NaN
...,...,...,...,...,...,...,...,...
3102,WY-037,Sweetwater County,Wyoming,8526,-432,40,8958,NaN
3103,WY-039,Teton County,Wyoming,10747,-392,32,11139,NaN
3104,WY-041,Uinta County,Wyoming,8974,-463,30,9437,NaN
3105,WY-043,Washakie County,Wyoming,7590,-618,23,8208,NaN


In [74]:
unique_values = msa_county['State'].unique()
unique_values

array(['TX', 'OH', 'GA', 'OR', 'NY', 'NM', 'LA', 'NJ', 'PA', 'IA', 'AK',
       'MI', 'AL', 'WI', 'NC', 'SC', 'CA', 'MD', 'ME', 'MA', 'WV', 'WA',
       'MT', 'ND', 'VA', 'IL', 'IN', 'ID', 'NH', 'CO', 'KY', 'CT', 'VT',
       'FL', 'MO', 'NV', 'WY', 'TN', 'DE', 'MN', 'OK', 'AR', 'AZ', 'NE',
       'MS', 'HI', 'KS', 'UT', 'RI', 'SD', 'DC'], dtype=object)

In [75]:
msa_county[msa_county['State'] == 'DC']

,County Name,State,MSA
1114,District of Columbia,DC,"Washington-Arlington-Alexandria, DC-VA-MD-WV"


In [76]:
HDD

unique_values = HDD['State'].unique()
unique_values

array(['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', ' D.C.', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [77]:
# Dictionary to map full state names to state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', ' D.C.': 'DC'
}

# Step 1: Clean the first dataframe by removing 'County' from the 'Name' column and converting state names to abbreviations
HDD['Name'] = HDD['Name'].str.replace(' County', '', regex=False)
HDD['State'] = HDD['State'].apply(lambda x: state_abbreviations.get(x, x))

In [78]:
merged_data = pd.merge(HDD, msa_county, left_on=['Name', 'State'], right_on=['County Name', 'State'], how='left')

In [79]:
missing_states = merged_data[merged_data['County Name'].isna()]
missing_states

,ID,Name,State,Value,Anomaly (1901-2000 base period),Rank,1901-2000 Mean,Unnamed: 7,County Name,MSA
2,AL-005,Barbour,AL,1788,-436,9,2224,NaN,NaN,NaN
5,AL-011,Bullock,AL,1846,-521,7,2367,NaN,NaN,NaN
6,AL-013,Butler,AL,1646,-623,3,2269,NaN,NaN,NaN
8,AL-017,Chambers,AL,2411,-497,9,2908,NaN,NaN,NaN
9,AL-019,Cherokee,AL,2640,-612,7,3252,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3107,WY-037,Sweetwater,WY,8526,-432,40,8958,NaN,NaN,NaN
3108,WY-039,Teton,WY,10747,-392,32,11139,NaN,NaN,NaN
3109,WY-041,Uinta,WY,8974,-463,30,9437,NaN,NaN,NaN
3110,WY-043,Washakie,WY,7590,-618,23,8208,NaN,NaN,NaN


In [80]:
# Assuming your DataFrame is named df
merged_data = merged_data.dropna(subset=['MSA'])
merged_data

,ID,Name,State,Value,Anomaly (1901-2000 base period),Rank,1901-2000 Mean,Unnamed: 7,County Name,MSA
0,AL-001,Autauga,AL,1862,-594,5,2456,NaN,Autauga,"Montgomery, AL"
1,AL-003,Baldwin,AL,1293,-472,4,1765,NaN,Baldwin,"Daphne-Fairhope-Foley, AL"
3,AL-007,Bibb,AL,2272,-481,11,2753,NaN,Bibb,"Birmingham-Hoover, AL"
4,AL-009,Blount,AL,2585,-646,8,3231,NaN,Blount,"Birmingham-Hoover, AL"
7,AL-015,Calhoun,AL,2456,-666,5,3122,NaN,Calhoun,"Anniston-Oxford, AL"
...,...,...,...,...,...,...,...,...,...,...
3083,WI-131,Washington,WI,7224,-654,21,7878,NaN,Washington,"Milwaukee-Waukesha, WI"
3084,WI-133,Waukesha,WI,6813,-748,16,7561,NaN,Waukesha,"Milwaukee-Waukesha, WI"
3087,WI-139,Winnebago,WI,7115,-927,13,8042,NaN,Winnebago,"Oshkosh-Neenah, WI"
3099,WY-021,Laramie,WY,6962,-812,12,7774,NaN,Laramie,"Cheyenne, WY"


In [81]:
merged_data['Value'] = pd.to_numeric(merged_data['Value'])

grouped_msa_sector = merged_data.groupby(['MSA'])['Value'].sum().reset_index()
grouped_msa_sector.rename(columns={'Value': 'Heating Degree Days'}, inplace=True)
grouped_msa_sector

,MSA,Heating Degree Days
0,"Abilene, TX",6996
1,"Akron, OH",10676
2,"Albany, GA",6104
3,"Albany-Lebanon, OR",5550
4,"Albany-Schenectady-Troy, NY",31836
...,...,...
364,"Yakima, WA",6455
365,"York-Hanover, PA",4662
366,"Youngstown-Warren-Boardman, OH-PA",17086
367,"Yuba City, CA",4676


In [82]:
#take the average value of the summation of heating degree days
result = merged_data.groupby('MSA')['Value'].sum() / merged_data.groupby('MSA')['County Name'].count()

In [83]:
result

MSA
Abilene, TX                          2332.000000
Akron, OH                            5338.000000
Albany, GA                           1526.000000
Albany-Lebanon, OR                   5550.000000
Albany-Schenectady-Troy, NY          6367.200000
                                        ...     
Yakima, WA                           6455.000000
York-Hanover, PA                     4662.000000
Youngstown-Warren-Boardman, OH-PA    5695.333333
Yuba City, CA                        2338.000000
Yuma, AZ                             1146.000000
Length: 369, dtype: float64

In [84]:
#to understand the climate condition -using the average
#to understand the energy demand or the total needs - using sum 

In [85]:
# Convert the Series to DataFrame
result = result.reset_index()
result.columns = ['MSA', 'Heating Degree Days']
result

,MSA,Heating Degree Days
0,"Abilene, TX",2332.000000
1,"Akron, OH",5338.000000
2,"Albany, GA",1526.000000
3,"Albany-Lebanon, OR",5550.000000
4,"Albany-Schenectady-Troy, NY",6367.200000
...,...,...
364,"Yakima, WA",6455.000000
365,"York-Hanover, PA",4662.000000
366,"Youngstown-Warren-Boardman, OH-PA",5695.333333
367,"Yuba City, CA",2338.000000


In [87]:
result.to_csv('../../data/tidy/heating-degree-days-average.csv', index=False)
grouped_msa_sector.to_csv('../../data/tidy/heating-degree-days-sum.csv', index=False)